In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import r2_score
%matplotlib inline

In [2]:
df1=pd.read_excel('/home/ayushi/Downloads/RE/R1.xlsx',sheet_name='Sheet1', header=1)

In [3]:
df1.head()

,Calendar Year Month,Product Line Desc (M),Product Class Desc (M),Sales District Desc (M),ABC indicator,Unnamed: 5,USD,USD.1,USD.2,USD.3,USD.4,USD.5,Unnamed: 12
0,201701,Arresters,#,North East Region,##,0.00,0.0,0.00,0.0,536.34,0.0,0.0,536.34
1,201701,Arresters,Blocks,EuropeMidEastAfrica,B,0.00,0.0,0.00,0.0,23040.00,23040.0,15360.0,61440.00
2,201701,Arresters,Blocks,EuropeMidEastAfrica,C,60588.00,60588.0,0.00,0.0,25486.20,46656.0,190256.0,322986.20
3,201701,Arresters,Blocks,North East Region,C,0.00,0.0,0.00,0.0,5691.00,0.0,0.0,5691.00
4,201701,Arresters,Distribution Arresters,APAC,1,9963.52,0.0,9963.52,0.0,0.00,0.0,0.0,9963.52


In [5]:
df1 = df1.iloc[:-1] # deleting last row

df1['Calendar Year Month']=pd.to_datetime(df1['Calendar Year Month'],format='%Y%m')
df1['year']=df1['Calendar Year Month'].apply(lambda x: x.year) 
df1['month']=df1['Calendar Year Month'].apply(lambda x: x.month)
df1.drop(['Calendar Year Month'],axis=1,inplace=True)

In [9]:
def cleaning(x):
    a=df1[df1['Product Line Desc (M)']==x]
    a.drop(['Product Line Desc (M)'],axis=1,inplace=True)
    le = preprocessing.LabelEncoder()
    a['Sales District Desc (M) '] = le.fit_transform(a['Sales District Desc (M) '])
    one_hot1 = pd.get_dummies(a['Product Class Desc (M)'])
    one_hot2 = pd.get_dummies(a['ABC indicator'])
    a = pd.concat([a, one_hot1], axis=1)
    a=pd.concat([a,one_hot2], axis=1)
    a.drop(['Product Class Desc (M)','ABC indicator'],axis=1,inplace=True)
    a.drop(['USD','USD.1','USD.2','USD.3','USD.4','USD.5','Unnamed: 12'],inplace=True,axis=1)
    return a

In [10]:
cable=cleaning('Cable Accessories')
cable.head()

/home/ayushi/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Sales District Desc (M),Unnamed: 5,year,month,URD Accessories,URD Arresters,1,A,B,C,D
181,1,2791.66,2017,1,1,0,0,1,0,0,0
182,1,5512.37,2017,1,1,0,0,0,1,0,0
183,1,0.00,2017,1,1,0,0,0,0,0,1
184,3,0.00,2017,1,1,0,0,1,0,0,0
185,3,0.00,2017,1,1,0,0,0,1,0,0


In [11]:
Y=cable['Unnamed: 5']
cable.drop('Unnamed: 5',axis=1,inplace=True)

In [12]:
cable.head()

,Sales District Desc (M),year,month,URD Accessories,URD Arresters,1,A,B,C,D
181,1,2017,1,1,0,0,1,0,0,0
182,1,2017,1,1,0,0,0,1,0,0
183,1,2017,1,1,0,0,0,0,0,1
184,3,2017,1,1,0,0,1,0,0,0
185,3,2017,1,1,0,0,0,1,0,0


In [13]:
X=cable

In [14]:
X.dtypes

Sales District Desc (M)     int64
year                        int64
month                       int64
URD Accessories             uint8
URD Arresters               uint8
1                           uint8
A                           uint8
B                           uint8
C                           uint8
D                           uint8
dtype: object

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [144]:
model = svm.SVR(C=800000,epsilon=100,kernel='rbf')
# model = svm.SVR(C=1,epsilon=1,kernel='rbf')

In [145]:
model.fit(X_train,y_train)

SVR(C=800000, cache_size=200, coef0=0.0, degree=3, epsilon=100, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [146]:
predictions=model.predict(X_test)

In [147]:
r2_score(y_test,predictions)

0.27785708057307124